In [ ]:
import numpy as np
import os
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import ImageDataGenerator

from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.vgg16 import preprocess_input,decode_predictions

เข้าถึง dataset

In [ ]:
test_fd = 'dataset_03'

from google.colab import drive
drive.mount('/content/drive')
path = '/content/drive/MyDrive/Colab Notebooks/' + test_fd 
print(path)
os.listdir(path)

train_dir = path + '/train_set'
val_dir = path + '/val_set'
test_dir = path + '/test_set'

target_img_shape=(128,128)

In [ ]:
train_datagen = ImageDataGenerator(preprocessing_function=preprocess_input,
                                   rotation_range=20, # +-
                                   height_shift_range=0.15, # + -
                                   width_shift_range=0.15, # + -
                                   shear_range=0.9,
                                   zoom_range=0.2,
                                   horizontal_flip=True)
train_set = train_datagen.flow_from_directory(train_dir,
                                              target_size=target_img_shape,
                                              batch_size=32,
                                              class_mode='sparse')

val_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)
val_set = val_datagen.flow_from_directory(val_dir,
                                          target_size=target_img_shape,
                                          batch_size=32,
                                          class_mode='sparse')

test_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)
test_set = test_datagen.flow_from_directory(test_dir,
                                            target_size=target_img_shape,
                                            batch_size=32,
                                            class_mode='sparse',
                                            shuffle=False)

จำนวนภาพที่ใช้ในการ train, val และ test

In [ ]:
labels = (train_set.class_indices)
labels = dict((v,k) for k,v in labels.items())
print(labels)

print('Train')
ids, counts = np.unique(train_set.classes,return_counts=True)
for i in ids:
    print(labels[i]+' = '+str(counts[i]))

print('Val')
ids, counts = np.unique(val_set.classes,return_counts=True)
for i in ids:
    print(labels[i]+' = '+str(counts[i]))

print('Test')
ids, counts = np.unique(test_set.classes,return_counts=True)
for i in ids:
    print(labels[i]+' = '+str(counts[i]))

input_shape = (target_img_shape[0],target_img_shape[1],3)
print(input_shape)

โครงสร้างของ model

In [ ]:
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Flatten, Dense

base_model = VGG16(include_top=False,
                   weights='imagenet',
                   input_shape=input_shape)
base_model.summary()

model = Sequential()

model.add(base_model)

model.add(Flatten())
model.add(Dense(128,activation='relu'))
model.add(Dense(128,activation='relu'))
model.add(Dense(3,activation='softmax'))

model.summary()

base_model.trainable = True # ทำให้ทุกชั้น train ได้

for layer in base_model.layers[:-12]: # เฉพาะ 12 ชั้นท้าย train ได้
    layer.trainable = False

for variable in model.trainable_variables:
    print(variable.name)

#from tensorflow.keras.utils import plot_model
#plot_model(model,show_shapes=True,show_layer_names=True)

train model

In [ ]:
from tensorflow.keras.optimizers import Adam
opt = Adam(learning_rate=1e-5)
model.compile(loss='sparse_categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])

history = model.fit(train_set,
                    validation_data=val_set,
                    epochs=50,
                    verbose=1)

แสดงกราฟ loss และ accuracy

In [ ]:
plt.figure()
plt.title('Loss')
plt.plot(history.history['loss'],'b',label='Training Loss')
plt.plot(history.history['val_loss'],'r',label='Validation Loss')
plt.legend()
plt.show()

plt.figure()
plt.title('Accuracy')
plt.plot(history.history['accuracy'],'b',label='Training Accuracy')
plt.plot(history.history['val_accuracy'],'r',label='Validation Accuracy')
plt.legend()
plt.show()

ทดสอบกับภาพ 1 ภาพ

In [ ]:
from tensorflow.keras.preprocessing.image import load_img, img_to_array

#test_image = load_img(test_dir+'/rose/'+'6125332325_b768e08bd9_n.jpg',target_size=target_img_shape)
#test_image = load_img(test_dir+'/sunflower/'+'5180859236_60aa57ff9b_n.jpg',target_size=target_img_shape)
test_image = load_img(test_dir+'/tulip/'+'5704726114_a92f753514.jpg',target_size=target_img_shape)
plt.imshow(test_image)
plt.show()

test_image = img_to_array(test_image)
test_image = preprocess_input(test_image)

test_image = np.expand_dims(test_image,axis=0) # (1, 128, 128, 3)
print(test_image.shape)

result = model.predict(test_image)
print(result)
print(train_set.class_indices)

class_answer = np.argmax(result,axis=1)
if class_answer == 0:
    predict = 'rose'
elif class_answer == 1:
    predict = 'sunflower'
elif class_answer == 2:
    predict = 'tulip'
print(predict)

ทดสอบกับภาพทั้ง folder test_set

In [ ]:
acc = model.evaluate(test_set)
print('accuracy_score =',acc[1])

In [ ]:
#Confution Matrix and Classification Report
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

Y_pred = model.predict(test_set)
print(len(Y_pred))

y_pred = np.argmax(Y_pred,axis=1)

print('test_set.classes = ',test_set.classes)
print('y_pred = ', y_pred)

print('Confusion Matrix')
print(confusion_matrix(test_set.classes, y_pred))
print('Classification Report')
target_names = ['rose', 'sunflower','tulip']
print(classification_report(test_set.classes, y_pred, target_names=target_names))
print('accuracy_score = ',accuracy_score(test_set.classes, y_pred))

import seaborn as sns
ax = plt.subplot()
cm = confusion_matrix(test_set.classes,y_pred)
sns.heatmap(cm,annot=True,ax=ax,cmap='coolwarm')
ax.set_xlabel('Predicted labels')
ax.set_ylabel('Actual labels')
ax.set_title('Confusion Matrix')
ax.xaxis.set_ticklabels(target_names)
ax.yaxis.set_ticklabels(target_names)
plt.show() 